### Custom Layers

In [5]:
import tensorflow as tf
import numpy as np
from keras import backend as K
from tensorflow.keras.losses import Loss
from tensorflow.keras.layers import Layer
from tensorflow.keras.layers import Dense, Lambda, Flatten

In [33]:
class SimpleDense(Layer):
    
    def __init__(self, units=32):
        super(SimpleDense, self).__init__()
        self.units = units
        
    def build(self, input_shape):
        w_init = tf.random_normal_initializer()
        self.w = tf.Variable(name="kernel",
                             initial_value = w_init(shape=(input_shape[-1], self.units), dtype='float32'),
                             trainable = True)
        b_init = tf.zeros_initializer()
        self.b = tf.Variable(name="bias",
                             initial_value = b_init(self.units), dtype='float32', trainable=True)
        
    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b
        
my_dense = SimpleDense(units=1)
x = tf.ones((1,1))
y = my_dense(x)
print(my_dense.variables)

[<tf.Variable 'simple_dense_15/kernel:0' shape=(1, 1) dtype=float32, numpy=array([[-0.025837]], dtype=float32)>, <tf.Variable 'simple_dense_15/bias:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>]


In [36]:
xs = np.array([-1.0,  0.0, 1.0, 2.0, 3.0, 4.0], dtype='float32')
ys = np.array([-3.0, -1.0, 1.0, 3.0, 5.0, 7.0], dtype='float32')

my_layer = SimpleDense(units=1)
model = tf.keras.Sequential([my_layer])
model.compile(optimizer='sgd', loss='mean_squared_error')
model.fit(xs, ys, epochs=500, verbose=0)
print(model.predict([10]))
print(my_layer.variables)

[[18.981544]]
[<tf.Variable 'simple_dense_18/kernel:0' shape=(1, 1) dtype=float32, numpy=array([[1.9973252]], dtype=float32)>, <tf.Variable 'simple_dense_18/bias:0' shape=(1,) dtype=float32, numpy=array([-0.9917073], dtype=float32)>]


### Custom Layers with Activation

In [55]:
from tensorflow.keras.layers import Layer
from tensorflow.keras.losses import Loss
from tensorflow.keras.layers import Lambda, Dense, Input
from tensorflow.keras.models import Model
from keras import backend as  K

class SimpleDenseAct(Layer):
    
    def __init__(self, units=32, activation = None):
        super(SimpleDenseAct, self).__init__()
        self.units = units
        self.activation = tf.keras.activations.get(activation)
        
    def build(self, input_shape):
        w_init = tf.random_normal_initializer()
        
        self.w = tf.Variable(name = "kernel",
                             initial_value = w_init(shape=(input_shape[-1], self.units), dtype='float32'),
                             trainable=True)
        b_init = tf.zeros_initializer()

        self.b = tf.Variable(name = "bias",
                             initial_value = b_init(self.units, dtype='float32'),
                             trainable=True)
        super().build(input_shape)
        
    def call(self, inputs):
        return self.activation(tf.matmul(inputs, self.w) + self.b)

    
class HuberLoss(Loss):
    
    def __init__(self, threshold=1):
        super(HuberLoss, self).__init__()
        self.threshold = threshold
    
    def call(self, y_true, y_pred):
        error = y_true - y_pred
        condition = K.abs(error) <= threshold
        small_error = K.square(error) / 2
        large_error = threshold * (K.abs(error) - 0.5* threshold)
        return tf.where(condition, small_error, large_error)

    
def relu_layer(x):
    return K.max(0, x)

In [56]:
input_layer = Input(shape=(8,))
my_layer = SimpleDenseAct(units=32)(input_layer)
last_layer = Dense(1)(my_layer)
model  = Model(inputs=input_layer, outputs=last_layer)

In [57]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 8)]               0         
_________________________________________________________________
simple_dense_act_4 (SimpleDe (None, 32)                288       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 33        
Total params: 321
Trainable params: 321
Non-trainable params: 0
_________________________________________________________________
